In [1]:
import pandas as pd
import numpy as np
import geopandas
import psycopg2
import infostop
import datetime
import time
import os
import random
import logging

In [ ]:
'https://drive.google.com/file/d/1tMnQghLu7K31ngsgWa_dVxufKNpx_vNG/view?usp=sharing'

In [12]:
!gdown --id 1tMnQghLu7K31ngsgWa_dVxufKNpx_vNG

/bin/bash: gdown: command not found


In [2]:
import logging
logger = logging.getLogger('locations_log')
logger.setLevel(logging.DEBUG)
fh = logging.FileHandler('locations.log', mode='w')
fh.setLevel(logging.DEBUG)
logger.addHandler(fh)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
logger.addHandler(fh)


In [56]:
tmp = pd.read_csv("Data/location_202009.csv")

In [57]:
tmp.shape

(5481780, 4)

In [60]:
files = os.listdir("Data/devices/")
len(files)

1907678

In [10]:
path = "Data/devices_list/"
files = os.listdir(path)
len(files)

527522

In [59]:
1326/60

22.1

In [51]:
f = 'FILE536a2d39-3988-4fe3-a7dc-f9cb14729ec4.csv'
f in files

True

In [52]:
x = pd.read_csv(os.path.join("Data/devices/", f)).sort_values('datetime')
# cond = x['datetime'] >= '2020-05-27 07:26:19'
x

,device_id,datetime,x,y
9525,536a2d39-3988-4fe3-a7dc-f9cb14729ec4,2020-04-04 00:25:55,-75.202864,-12.090289
9524,536a2d39-3988-4fe3-a7dc-f9cb14729ec4,2020-04-04 16:20:27,-75.203097,-12.089986
9432,536a2d39-3988-4fe3-a7dc-f9cb14729ec4,2020-04-07 13:54:49,-75.203007,-12.090346
9434,536a2d39-3988-4fe3-a7dc-f9cb14729ec4,2020-04-08 20:08:24,-75.202863,-12.090286
9435,536a2d39-3988-4fe3-a7dc-f9cb14729ec4,2020-04-08 20:08:26,-75.202863,-12.090286
...,...,...,...,...
2687,536a2d39-3988-4fe3-a7dc-f9cb14729ec4,2020-05-31 23:43:03,-75.203061,-12.090026
2914,536a2d39-3988-4fe3-a7dc-f9cb14729ec4,2020-05-31 23:49:04,-75.202923,-12.090207
2911,536a2d39-3988-4fe3-a7dc-f9cb14729ec4,2020-05-31 23:54:26,-75.202924,-12.090279
8564,536a2d39-3988-4fe3-a7dc-f9cb14729ec4,2020-05-31 23:54:26,-75.202924,-12.090279


In [53]:
x.sort_index()

,device_id,datetime,x,y
0,536a2d39-3988-4fe3-a7dc-f9cb14729ec4,2020-05-29 04:50:03,-75.203017,-12.090325
1,536a2d39-3988-4fe3-a7dc-f9cb14729ec4,2020-05-29 03:56:45,-75.203005,-12.090326
2,536a2d39-3988-4fe3-a7dc-f9cb14729ec4,2020-05-29 05:59:07,-75.203016,-12.090317
3,536a2d39-3988-4fe3-a7dc-f9cb14729ec4,2020-05-29 04:00:04,-75.203025,-12.090253
4,536a2d39-3988-4fe3-a7dc-f9cb14729ec4,2020-05-29 04:10:20,-75.203277,-12.089986
...,...,...,...,...
9521,536a2d39-3988-4fe3-a7dc-f9cb14729ec4,2020-04-18 07:20:46,-75.202887,-12.090685
9522,536a2d39-3988-4fe3-a7dc-f9cb14729ec4,2020-04-18 07:34:37,-75.202954,-12.090063
9523,536a2d39-3988-4fe3-a7dc-f9cb14729ec4,2020-04-18 08:30:17,-75.203099,-12.089974
9524,536a2d39-3988-4fe3-a7dc-f9cb14729ec4,2020-04-04 16:20:27,-75.203097,-12.089986


In [3]:
def process(df):
    ts = time.time()
    i = 0
    logger.info("File size {}: ".format(df.shape) )
    users = df.index.unique()
    logger.info("Total unique users: %s", len(users))
    for device in users:
        name = "{}/FILE{}.csv".format(path, device)
        if not os.path.isfile(name):
            df.loc[device].to_csv(name, index=False)
        else:
            df.loc[device].to_csv(name, mode='a', header=False)
#         print("{} Writing on {} file".format(i, name))
        i = i + 1
        if i % 100 == 0:
            logger.info('i: %s users took %s seconds', i, time.time() - ts)

In [9]:
files = os.listdir('Data/devices2')
files[:10]

['FILEb54f42d0-44d0-461f-bb56-4c2505952b5c.csv',
 'FILEa3591ccf-873c-45b7-9373-1f08e31f8461.csv',
 'FILEac93391f-76bc-489e-8ca3-958fb21a7037.csv',
 'FILE38b86ea9-65c0-4e36-a853-6cc91ce610d2.csv',
 'FILE15c235d3-fb23-4d69-911f-de2c1f0c6a4b.csv',
 'FILEdeadd1eb-979d-4112-9312-ddd66651e70a.csv',
 'FILE9b38022e-6baf-4529-be86-e3ee6b85dea5.csv',
 'FILE2cee0a48-3004-4350-83bb-4ad380c2d745.csv',
 'FILEe67aed1c-6e76-407d-b20f-f1a13a0f42de.csv',
 'FILEc2676dfa-f3af-47c2-9bb2-a578de048da4.csv']

In [33]:
# tmp = pd.read_csv('Data/devices2/FILEa3591ccf-873c-45b7-9373-1f08e31f8461.csv')

# file = open('Data/devices2/FILEa3591ccf-873c-45b7-9373-1f08e31f8461.csv')
with open('Data/devices/FILE2cee0a48-3004-4350-83bb-4ad380c2d745.csv') as reader:
    # Further file processing goes here
    print(reader.read())


device_id,datetime,x,y
2cee0a48-3004-4350-83bb-4ad380c2d745,2020-05-28 23:53:35,-77.0479955,-12.0617727
2cee0a48-3004-4350-83bb-4ad380c2d745,2020-05-19 00:00:08,-77.0796153,-11.9492238
2cee0a48-3004-4350-83bb-4ad380c2d745,2020-05-19 00:25:22,-77.0796265,-11.949195800000002
2cee0a48-3004-4350-83bb-4ad380c2d745,2020-05-19 01:32:37,-77.07961309999997,-11.9492199
2cee0a48-3004-4350-83bb-4ad380c2d745,2020-05-19 00:17:50,-77.0795987,-11.948866699999998
2cee0a48-3004-4350-83bb-4ad380c2d745,2020-05-19 00:15:20,-77.0796221,-11.9492335
2cee0a48-3004-4350-83bb-4ad380c2d745,2020-05-19 00:10:16,-77.0796207,-11.9492232
2cee0a48-3004-4350-83bb-4ad380c2d745,2020-05-19 00:07:45,-77.0796218,-11.9492222
2cee0a48-3004-4350-83bb-4ad380c2d745,2020-05-19 01:29:38,-77.0796164,-11.9492179
2cee0a48-3004-4350-83bb-4ad380c2d745,2020-05-19 01:33:26,-77.079622,-11.9492235



In [36]:
pd.read_csv('Data/devices/FILEa3591ccf-873c-45b7-9373-1f08e31f8461.csv')
# part.loc['2cee0a48-3004-4350-83bb-4ad380c2d745']

,device_id,datetime,x,y
0,a3591ccf-873c-45b7-9373-1f08e31f8461,2020-05-28 14:39:28,-77.090399,-12.030942
1,a3591ccf-873c-45b7-9373-1f08e31f8461,2020-05-28 12:54:33,-77.090346,-12.030982
2,a3591ccf-873c-45b7-9373-1f08e31f8461,2020-05-28 14:43:29,-77.090395,-12.030943
3,a3591ccf-873c-45b7-9373-1f08e31f8461,2020-05-28 21:51:04,-77.090395,-12.030784
4,a3591ccf-873c-45b7-9373-1f08e31f8461,2020-05-28 19:44:54,-77.090400,-12.030945
...,...,...,...,...
354,a3591ccf-873c-45b7-9373-1f08e31f8461,2020-05-28 14:06:46,-77.090446,-12.030880
355,a3591ccf-873c-45b7-9373-1f08e31f8461,2020-05-28 16:41:21,-77.090454,-12.030823
356,a3591ccf-873c-45b7-9373-1f08e31f8461,2020-05-28 20:03:41,-77.090396,-12.030944
357,a3591ccf-873c-45b7-9373-1f08e31f8461,2020-05-10 13:03:14,-77.090451,-12.030659


In [30]:
import pandas as pd

df = pd.DataFrame(data=range(5), index=[1, 2, 3, 3, 3])

print(type(df.loc[[3]]))

print(type(df.loc[[1]]))

df.loc[3]

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


,0
3,2
3,3
3,4


In [51]:
%%time
# df = pd.read_csv('location_202005.csv.zip', compression='zip', nrows=1000000)
df = pd.read_csv('location_202005.csv.zip', compression='zip')
# (579044977, 4)

# CPU times: user 1h 53min 52s, sys: 12min 56s, total: 2h 6min 48s
# Wall time: 2h 10min 53s

CPU times: user 1h 53min 52s, sys: 12min 56s, total: 2h 6min 48s
Wall time: 2h 10min 53s


In [11]:
%%time
df.set_index('device_id', inplace=True)

In [99]:
df.shape
ts = time.time()
logging.info('Took %s seconds', time.time() - ts)
df.shape

2021-03-14 10:21:54,371 - root - INFO - Took 0.00016164779663085938 seconds


(579044977, 4)

In [145]:
df.shape

(579044977, 4)

In [56]:
df.dtypes

datetime      object
device_id     object
x            float64
y            float64
dtype: object

In [149]:
%%time
dfdict = df.groupby('device_id')[['datetime', 'x', 'y']].agg({'datetime':list, 'x':list, 'y':list})
dfdict

# 10000000
# CPU times: user 5min 3s, sys: 16.5 s, total: 5min 19s
# Wall time: 5min 36s

KeyboardInterrupt: 

In [147]:
dfuser

,datetime,x,y
device_id,,,
000044e3-5b64-415c-97c7-5ff91f35ded0,"[2020-05-10 20:59:56, 2020-05-10 21:05:31, 202...","[-77.05073279999998, -77.050736, -77.050706999...","[-12.0586779, -12.0586897, -12.0586641, -12.05..."
00005c55-aae7-48dc-82b2-08a56e02f792,"[2020-05-28 22:12:26, 2020-05-28 06:34:10, 202...","[-71.5496593, -71.5698705, -71.55015240000002,...","[-16.399632099999998, -16.4283434, -16.390094,..."
0000a268-a920-4605-b3a2-8e58c9053a8e,[2020-05-10 00:21:28],[-76.9426332],[-12.2429436]
0000bb86-fe13-425b-8dbd-adcbbc040d6f,"[2020-05-10 04:09:05, 2020-05-10 16:15:09, 202...","[-71.52163259999998, -71.5215337, -71.5213891,...","[-16.429250800000002, -16.429169199999997, -16..."
0000f455-5eb8-4469-9880-bb82a22993bb,"[2020-05-10 17:44:45, 2020-05-10 13:44:56, 202...","[-77.1134391, -77.11350300000002, -77.1133718,...","[-11.8664694, -11.8663435, -11.866353300000002..."
...,...,...,...
fffdf688-899d-45f2-9b2a-01bab28466ac,"[2020-05-28 21:41:48, 2020-05-28 19:01:21, 202...","[-77.0121533, -77.0120168, -77.0120089, -77.01...","[-12.2072649, -12.207049, -12.2070456, -12.207..."
fffe1608-95aa-4ad7-a78f-fd2e99a68859,[2020-05-19 23:17:20],[-79.90203100000002],[-6.7037701]
fffe63d9-4285-4efb-a329-2c9d3f9b4f64,"[2020-05-19 21:53:58, 2020-05-19 22:33:52, 202...","[-77.1139936, -77.11403890000003, -77.1139964,...","[-12.0457341, -12.0458658, -12.0457354, -12.04..."


In [89]:
dfsample.to_csv("location_sample.csv", index=False)

In [90]:
pd.read_csv("xaa")

,datetime,device_id,x,y
0,2020-05-28 00:22:29,0398b16b-7513-46f8-99d8-e7195301defc,-77.005935,-12.104188
1,2020-05-28 00:40:21,e7ae66ca-0aa0-4417-b580-992ba3d44128,-77.036477,-11.902131
2,2020-05-28 00:26:09,ef849e19-c9aa-4c92-b9c3-aff28e8b82a6,-77.048523,-12.093954
3,2020-05-28 00:18:01,d6d9be9d-989b-4e83-a52a-4366d29afe1a,-81.257206,-4.580383
4,2020-05-28 00:36:49,ed6e6fc5-5b58-4ac0-becb-33ffd5aeaba6,-76.971267,-12.022589
...,...,...,...,...
1999969,2020-05-28 01:19:03,4867803c-d353-43eb-a6ef-1c5466095e22,-77.021456,-12.156008
1999970,2020-05-28 01:18:46,d51cb10a-50ef-4f92-ba4c-89e4f1aa4177,-77.058354,-11.935601
1999971,2020-05-28 00:55:37,51d43799-61a9-45d6-b8e7-1dbfc52ff6bf,-77.053329,-11.923541
1999972,2020-05-28 01:04:34,50e808f8-2c1d-445e-8167-9961497d42bd,-76.984629,-12.078509


In [13]:
process(df)

0 Writing on location202005/FILE0398b16b-7513-46f8-99d8-e7195301defc.csv file
1 Writing on location202005/FILEe7ae66ca-0aa0-4417-b580-992ba3d44128.csv file
2 Writing on location202005/FILEef849e19-c9aa-4c92-b9c3-aff28e8b82a6.csv file
3 Writing on location202005/FILEd6d9be9d-989b-4e83-a52a-4366d29afe1a.csv file
4 Writing on location202005/FILEed6e6fc5-5b58-4ac0-becb-33ffd5aeaba6.csv file
5 Writing on location202005/FILE5053ca01-a780-496c-9878-c6b04ad452f0.csv file
6 Writing on location202005/FILE9fbb7032-05be-4077-9791-20a33e9468e3.csv file
7 Writing on location202005/FILEa693c0ed-b4b9-4b17-bfc9-03d85bf74bb8.csv file
8 Writing on location202005/FILEbc1f2263-f408-4bec-b93e-e30d8f015370.csv file
9 Writing on location202005/FILE51949a68-d713-4728-8809-074e1526322a.csv file
10 Writing on location202005/FILEf29823d2-b1e4-421e-9c93-d62203448809.csv file
11 Writing on location202005/FILE1e05bb52-6ee8-42fc-9700-5c69f74797fd.csv file
12 Writing on location202005/FILE425a6b02-9910-45c3-ae50-3c684

104 Writing on location202005/FILE53d56a17-51a6-4fce-941e-501e90ec5618.csv file
105 Writing on location202005/FILE89550141-c061-41e4-b84f-0fa37716fb4c.csv file
106 Writing on location202005/FILE1ce856b2-3df4-4f50-9d07-30f94a16dc4c.csv file
107 Writing on location202005/FILE45d89ada-508c-48ad-b3d4-143486e0450a.csv file
108 Writing on location202005/FILE470168b7-f4e6-4f94-8339-0b4975c17d01.csv file
109 Writing on location202005/FILEee59cf86-fed4-4204-847b-73027f22e772.csv file
110 Writing on location202005/FILE3d9ebb87-2eae-4dcc-b5dc-0857c0b81b39.csv file
111 Writing on location202005/FILE7cf9d886-8dc4-4616-97ce-92540d3809f6.csv file
112 Writing on location202005/FILEac31b49b-2aa7-4b13-a5bf-0c49514e414c.csv file
113 Writing on location202005/FILE0ad06c38-3fb4-4031-aded-2587d10757b1.csv file
114 Writing on location202005/FILEbcc08f22-4473-4abd-9af6-3e72fce1daa0.csv file
115 Writing on location202005/FILE79b2eaf4-0d52-4b19-a2fb-5dbfe9b74696.csv file
116 Writing on location202005/FILE7742f4

207 Writing on location202005/FILE1522091b-3238-4910-8b4f-610ba2d70efa.csv file
208 Writing on location202005/FILEeef38701-7ca7-4a26-a26d-ddf2bb4557ef.csv file
209 Writing on location202005/FILEb58f6d48-42e8-4e9f-8a97-424e46949731.csv file
210 Writing on location202005/FILE2c78fa0d-72a9-484d-acbb-4c526fa363ab.csv file
211 Writing on location202005/FILEfdcf83b4-cc11-4b4c-b844-243d38407528.csv file
212 Writing on location202005/FILE5568e563-51d4-4af8-bcb2-11f4e8c13bb5.csv file
213 Writing on location202005/FILE60d23729-8e04-4b2f-baad-475fcce8c3ce.csv file
214 Writing on location202005/FILEc8d35a83-bf9f-47d6-a2c5-07560aa1c7fe.csv file
215 Writing on location202005/FILEf9630c72-1abd-473d-903d-c7a35a2aa731.csv file
216 Writing on location202005/FILE74f7bb26-edda-48f8-8ee0-1b4aa2b5f0bf.csv file
217 Writing on location202005/FILE8ade8c3a-6254-4b28-91be-63ab14da1449.csv file
218 Writing on location202005/FILEdf909c7f-95bb-4bed-a1ee-a3a27205f80b.csv file
219 Writing on location202005/FILE78f4e7

310 Writing on location202005/FILE0f6bdea9-d552-4a6d-9b59-3d39e34c2486.csv file
311 Writing on location202005/FILEbc16c176-c94e-4057-8cb8-d6af92a124f3.csv file
312 Writing on location202005/FILE7db7a279-ed5f-4386-929c-69bcf266e427.csv file
313 Writing on location202005/FILEc6953b3a-39f8-4304-b3f1-d49db812944d.csv file
314 Writing on location202005/FILE1f50f9ae-cf48-44cc-a321-df64604bc18c.csv file
315 Writing on location202005/FILE7e7371f8-8c7d-4df3-bc16-81139665fcb6.csv file
316 Writing on location202005/FILEcbdd1bef-61fb-4829-aa04-6705e687cfed.csv file
317 Writing on location202005/FILE86627e1d-23bc-4036-ae4c-f45f93048321.csv file
318 Writing on location202005/FILE8c855350-5ffd-4765-bd84-c40d70e3c524.csv file
319 Writing on location202005/FILE6ca103d8-1210-4e84-bc1e-d651d1f2dde1.csv file
320 Writing on location202005/FILE32b90244-3375-40a0-a423-2b302d11dedf.csv file
321 Writing on location202005/FILEca7ba4f0-8ab2-4d82-b72c-e54605146f31.csv file
322 Writing on location202005/FILE19f747

413 Writing on location202005/FILEf5cbfd61-46a7-43c2-8f53-7337edf97a54.csv file
414 Writing on location202005/FILEe1518f3c-3fb6-46cf-871b-55cc55031706.csv file
415 Writing on location202005/FILEb8f70fe4-aca8-4f74-a0fe-d21ad8ab4036.csv file
416 Writing on location202005/FILE5dec6fbc-40a2-4c37-ba9a-62dc72b6625c.csv file
417 Writing on location202005/FILE360ae822-562a-4a21-a925-1a2c20d866a0.csv file
418 Writing on location202005/FILE5236c1b1-5407-4735-9c0b-29d69e02287d.csv file
419 Writing on location202005/FILE06eb4f59-8fa7-4b47-b599-71bc2772d731.csv file
420 Writing on location202005/FILEb670a88c-7157-409a-b317-e7f9162aef8a.csv file
421 Writing on location202005/FILE2c34051c-e2c5-4584-bb35-91cedfa628cf.csv file
422 Writing on location202005/FILE33ea1713-a6b7-4e93-931b-e5ff7b3f59ae.csv file
423 Writing on location202005/FILEb9e01709-e30f-49bf-b7f3-fa772097a5af.csv file
424 Writing on location202005/FILEa5b4f0ce-89d0-4ffb-b378-68a26ab5c821.csv file
425 Writing on location202005/FILEc74d18

...

3077 Writing on location202005/FILE2f197546-b9e0-42d0-93b1-f9bd8b921811.csv file
3078 Writing on location202005/FILE9572015b-f22e-4b2f-b2a1-373b1f1055a3.csv file
3079 Writing on location202005/FILE773c9b39-3812-4e2b-9d64-0829ded3da1f.csv file
3080 Writing on location202005/FILE2925c38f-2567-4f92-8b12-af71f9a9799e.csv file
3081 Writing on location202005/FILE813c3cce-7063-40da-a6d8-968c322d07c9.csv file
3082 Writing on location202005/FILEa72f60be-07ae-407c-9109-2c838102e226.csv file
3083 Writing on location202005/FILE2ca04761-fd9e-408a-bcca-36f5dcedccc3.csv file
3084 Writing on location202005/FILEf0280571-7be0-4a10-978d-e41b680c08a7.csv file
3085 Writing on location202005/FILE74928fd8-ef65-44cf-9905-1bb891d06e98.csv file
3086 Writing on location202005/FILEa3d05f87-9934-4e40-80d9-9a630505b938.csv file
3087 Writing on location202005/FILE67a521b1-5766-4ca6-8197-406a194d947a.csv file
3088 Writing on location202005/FILE34771035-32d9-4734-aa67-364ed5f5e7c3.csv file
3089 Writing on location2020

3179 Writing on location202005/FILE3ccf423b-fb70-42cb-a462-8b2f1e13609b.csv file
3180 Writing on location202005/FILEad5f29b9-4faf-40c9-9e75-64557952d24a.csv file
3181 Writing on location202005/FILE71513982-826e-457e-945f-dacf32c8e78b.csv file
3182 Writing on location202005/FILE01d3166c-79fe-4fd3-94f3-10fbe97fbf4c.csv file
3183 Writing on location202005/FILE92576e3c-cbd5-4b67-934c-258fe8482f2c.csv file
3184 Writing on location202005/FILE1a5acd45-3c96-49a2-879f-76b4caff6878.csv file
3185 Writing on location202005/FILE8ce9adf7-f198-4343-b71c-a274dfb59037.csv file
3186 Writing on location202005/FILE16e36c30-718f-4a5a-9c3d-7862e7be0db1.csv file
3187 Writing on location202005/FILEb5360be7-8747-4225-90aa-1cabe6877644.csv file
3188 Writing on location202005/FILE09b6f8fb-7ff8-4b7d-9273-c269c5a908e2.csv file
3189 Writing on location202005/FILE5ed13cc2-f0dc-4db3-b106-e6b14d6ebf50.csv file
3190 Writing on location202005/FILE093b817c-cb1d-4200-9883-ddd0376f4e67.csv file
3191 Writing on location2020

3281 Writing on location202005/FILE055accb7-a02d-4bdf-bf98-119acc690369.csv file
3282 Writing on location202005/FILE902923af-a57d-4b88-a8cf-fc49c9325b8e.csv file
3283 Writing on location202005/FILE600e3bf8-f8d7-4934-bb44-f83844be1594.csv file
3284 Writing on location202005/FILEe1218b1f-f455-45de-bfa8-2ab38ff31536.csv file
3285 Writing on location202005/FILEff1fd0a9-3853-4d8e-a2f0-7ac6905169f3.csv file
3286 Writing on location202005/FILE61548bb1-be63-4f1b-a709-87933d490dc2.csv file
3287 Writing on location202005/FILE7bdaac19-7a45-4b74-9124-e3cac6e575b5.csv file
3288 Writing on location202005/FILE2320a060-4ae5-4606-9ac0-236c3c107884.csv file
3289 Writing on location202005/FILEb6bc97dc-6222-4068-b1db-d5e227bdee9f.csv file
3290 Writing on location202005/FILE1d688467-2e20-4e0c-8e3a-51c108ec7fb2.csv file
3291 Writing on location202005/FILE30edec2d-0128-4964-b635-bb10c3e6535c.csv file
3292 Writing on location202005/FILEf6ff02be-2d33-42a3-bbf4-6d1cf09559cf.csv file
3293 Writing on location2020

3383 Writing on location202005/FILE61720f22-1bd9-4005-9bf1-a8e71cc9fa1e.csv file
3384 Writing on location202005/FILEc301c4d0-c86b-4609-83b3-6306c42e5d3a.csv file
3385 Writing on location202005/FILE8c00a73a-c9ab-4e7b-bb72-3d29c626e32e.csv file
3386 Writing on location202005/FILEc50d5257-2454-4e82-9b08-63c72d9e56ca.csv file
3387 Writing on location202005/FILE1e443c0b-eb1e-4a7a-8300-d8a2a87ffa6b.csv file
3388 Writing on location202005/FILE91f0b3bb-eb45-461d-aab5-e5e58a3b883f.csv file
3389 Writing on location202005/FILE386fce54-b102-4115-b219-b5ccfdd79638.csv file
3390 Writing on location202005/FILE380922e4-77fb-452d-b34a-bb46344e4e64.csv file
3391 Writing on location202005/FILEfc92fe82-867c-482f-9d62-88e80dbd1bfc.csv file
3392 Writing on location202005/FILE860e91a8-f998-411d-acc1-0f1189050472.csv file
3393 Writing on location202005/FILE8ed3c6b6-6179-47a3-88a5-597cdb5f9d19.csv file
3394 Writing on location202005/FILE003fa37b-f5ea-4ec8-be1a-0495e516ff52.csv file
3395 Writing on location2020

KeyboardInterrupt: 

In [16]:
tmp = pd.read_csv("location202005/FILE00037f9a-0fde-4dee-bf86-c5450840a62d.csv")
tmp.head()

,datetime,x,y
0,2020-05-28 00:53:02,-77.049474,-11.995988
1,2020-05-28 00:53:02,-77.049485,-11.996041
2,2020-05-28 01:38:30,-77.049487,-11.996037
3,2020-05-28 00:53:01,-77.049465,-11.995973
4,2020-05-28 01:00:38,-77.049404,-11.996336


In [26]:
%%time
dfuser = df.reset_index().groupby('device_id')[['datetime', 'x', 'y']].agg({'datetime':list, 'x':list, 'y':list})
dfuser

CPU times: user 38.8 s, sys: 2.73 s, total: 41.5 s
Wall time: 44.7 s


,datetime,x,y
device_id,,,
00005c55-aae7-48dc-82b2-08a56e02f792,"[2020-05-28 22:12:26, 2020-05-28 06:34:10, 202...","[-71.5496593, -71.5698705, -71.55015240000002,...","[-16.399632099999998, -16.4283434, -16.390094,..."
00019dc8-a239-48b9-9933-545d501e2661,"[2020-05-28 01:22:56, 2020-05-28 00:54:19, 202...","[-76.9679399, -76.9679459, -76.9679458, -76.96...","[-12.0552451, -12.0552478, -12.0552475, -12.05..."
00037f9a-0fde-4dee-bf86-c5450840a62d,"[2020-05-28 00:53:02, 2020-05-28 00:53:02, 202...","[-77.0494735, -77.04948509999998, -77.0494871,...","[-11.9959884, -11.996041, -11.9960371, -11.995..."
00044f12-fb56-4fa5-b54f-3e4a6e0c3cd9,"[2020-05-28 17:13:23, 2020-05-28 20:03:20, 202...","[-71.8805854, -71.8803946, -71.88100159999998,...","[-13.547572099999998, -13.5504038, -13.5495773..."
00045606-3908-4015-a045-2b19a87aedb5,"[2020-05-28 01:43:57, 2020-05-28 01:48:28]","[-76.9070809, -76.9070821]","[-12.0283915, -12.0283908]"
...,...,...,...
fff986d1-5919-47ba-acc8-ee0d1de8063f,[2020-05-28 23:09:13],[-76.97851899999998],[-12.1445685]
fffaed3a-9ae6-4061-928b-bf091da8ae85,"[2020-05-28 20:01:05, 2020-05-28 19:57:44]","[-77.0543502, -77.05434509999998]","[-12.030627099999998, -12.0306629]"
fffb3a95-263c-4e11-bb5e-725ea404b124,"[2020-05-28 17:17:56, 2020-05-28 12:28:29, 202...","[-76.98886379999998, -76.98885809999999, -76.9...","[-12.1430302, -12.143012, -12.1429746, -12.143..."


In [28]:
dfuserdict = dfuser.to_dict('index')

In [148]:
len(dfuserdict)

74448

In [66]:
dfuserdict.get('fffaed3a-9ae6-4061-928b-bf091da8ae85')

{'datetime': ['2020-05-28 20:01:05', '2020-05-28 19:57:44'],
 'x': [-77.0543502, -77.05434509999998],
 'y': [-12.030627099999998, -12.0306629]}

In [77]:
tmp = {"187056f5-6344-49f0-917f-c8661ad5b418": {
        "datetime": [
            "2020-05-29 00:08:35"
        ],
        "x": [
            -77.0897824
        ],
        "y": [
            -12.0229049
        ]
    }}
tmp

{'187056f5-6344-49f0-917f-c8661ad5b418': {'datetime': ['2020-05-29 00:08:35'],
  'x': [-77.0897824],
  'y': [-12.0229049]}}

In [80]:
tmp[u]['datetime']

['2020-05-29 00:08:35']

In [78]:
u = '187056f5-6344-49f0-917f-c8661ad5b418'

dfuserdict[u]['datetime'].append(tmp[u]['datetime'])
dfuserdict[u]['x'].append(tmp[u]['x'])
dfuserdict[u]['y'].append(tmp[u]['y'])

In [79]:
dfuserdict['187056f5-6344-49f0-917f-c8661ad5b418']

{'datetime': ['2020-05-28 14:36:13', ['2020-05-29 00:08:35']],
 'x': [-77.04731190000003, [-77.0897824]],
 'y': [-12.0566907, [-12.0229049]]}

In [71]:
def sample_from_dict(d, sample=10):
    keys = random.sample(list(d), sample)
    values = [d[k] for k in keys]
    return dict(zip(keys, values))

In [74]:
import json

print(json.dumps(sample_from_dict(dfuserdict, 5), indent = 4))

{
    "187056f5-6344-49f0-917f-c8661ad5b418": {
        "datetime": [
            "2020-05-28 14:36:13"
        ],
        "x": [
            -77.04731190000003
        ],
        "y": [
            -12.0566907
        ]
    },
    "40d446f3-5f7c-4184-9174-e5b6cf8fb0d7": {
        "datetime": [
            "2020-05-28 00:03:59",
            "2020-05-28 05:19:43",
            "2020-05-28 15:08:25",
            "2020-05-28 15:38:27",
            "2020-05-28 03:16:53",
            "2020-05-28 16:24:40",
            "2020-05-28 04:12:41",
            "2020-05-28 15:16:16",
            "2020-05-28 12:47:23",
            "2020-05-28 15:29:12"
        ],
        "x": [
            -77.0666767,
            -77.06665699999998,
            -77.0337442,
            -77.03394449999998,
            -77.06666,
            -77.0337141,
            -77.0666526,
            -77.03375159999999,
            -77.0666493,
            -77.0337727
        ],
        "y": [
            -12.0890045,
          

In [25]:
df.loc['fffaed3a-9ae6-4061-928b-bf091da8ae85']

,datetime,x,y
device_id,,,
fffaed3a-9ae6-4061-928b-bf091da8ae85,2020-05-28 20:01:05,-77.054350,-12.030627
fffaed3a-9ae6-4061-928b-bf091da8ae85,2020-05-28 19:57:44,-77.054345,-12.030663


In [ ]:
filename = 'location_202005.csv.zip'

for chunk in pd.read_csv( filename, compression='zip', chunksize= 1000000):
    print(chunk.shape)

(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)
(1000000, 4)

In [ ]:
start_time = time.time()

chunksize = 10 ** 6

filename = 'location_202005.csv.zip'

i = 0
for chunk in pd.read_csv(filename,chunksize=1000000, compression='zip'):
    process(chunk)
    print("Bulk %s --- %s seconds ---" % (i , time.time() - start_time))
    i = i + 1

In [40]:
part = pd.read_csv("Data/location_part_00", nrows=1000)
part.head()

In [41]:
part.head()

,datetime,device_id,x,y
0,2020-05-28 00:22:29,0398b16b-7513-46f8-99d8-e7195301defc,-77.005935,-12.104188
1,2020-05-28 00:40:21,e7ae66ca-0aa0-4417-b580-992ba3d44128,-77.036477,-11.902131
2,2020-05-28 00:26:09,ef849e19-c9aa-4c92-b9c3-aff28e8b82a6,-77.048523,-12.093954
3,2020-05-28 00:18:01,d6d9be9d-989b-4e83-a52a-4366d29afe1a,-81.257206,-4.580383
4,2020-05-28 00:36:49,ed6e6fc5-5b58-4ac0-becb-33ffd5aeaba6,-76.971267,-12.022589


In [87]:
from multiprocessing import Process
import os

def info(title):
    print(title)
    print('module name:', __name__)
    print('parent process:', os.getppid())
    print('process id:', os.getpid())

def f(name):
    info('function f')
    print('hello', name)

if __name__ == '__main__':
    info('main line')
    p = Process(target=f, args=('bob',))
    p.start()
    p.join()

main line
module name: __main__
parent process: 20848
process id: 25325
function f
module name: __main__
parent process: 25325
process id: 28014
hello bob
